In [9]:
import datetime
import time
import pandas as pd
import yfinance as yf
pd.set_option('display.max_rows',None)
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy import stats


def move_download(TICKER):
    source = f'C:/Users/joaom/Downloads/{TICKER}.csv'
    destination = 'C:/Users/joaom/OneDrive/Documentos/GitHub/Snp1000/Performance Tables'
    dest = shutil.move(source, destination) 

companies = pd.read_csv('03_Tables/SnP1000_TICKR.csv')
companies_list = companies['Companie'].to_list()
df = pd.DataFrame()

start = datetime.datetime(2020,10,19)
end = datetime.datetime(2021,10,19)

#Module 1 - Get data
companies_list=['TSLA']
for i in companies_list[:1]:
    print(str(companies_list.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)
    try:
        stock = []
        stock = yf.download(i, start=start, end=end, progress=False)
        if len(stock) == 0:
            None
        else:
            stock['Name'] = i
            df = df.append(stock, sort=False)
    except Exception:
        None
        

FileNotFoundError: [Errno 2] No such file or directory: 'SNP1000/03_Tables/SnP1000_TICKR.csv'

In [ ]:
#Module 2 - Calculate RSI (relative Strength Index)
df = df.iloc[-60:] # Get just about the last two months of data

df.reset_index(inplace=True)
df['Change'] = (df['Close'].shift(-1)-df['Close']).shift(1)
df['ChangeAverage'] = df['Change'].rolling(window=2).mean()
df['ChangeAverage+'] = df.apply(lambda x: x['ChangeAverage'] if x['ChangeAverage'] > 0 else 0,axis=1).rolling(window=14).mean()
df['ChangeAverage-'] = df.apply(lambda x: x['ChangeAverage'] if x['ChangeAverage'] < 0 else 0,axis=1).rolling(window=14).mean()*-1
df['RSI'] = 100-(100/(1+(df['ChangeAverage+']/df['ChangeAverage-'])))
df = df.drop(columns = ['ChangeAverage','ChangeAverage','ChangeAverage'])


In [3]:
#Module 3 - Calculate OBV (On Balance Volume) 
VolumeList = df['Volume'].tolist()
OBV = [VolumeList[0]]
i=1
for i in range(1,len(VolumeList)):
    #i=i+1
    if VolumeList[i] > VolumeList[i-1]:
        OBV.append(OBV[i-1]+VolumeList[i])
        continue
    if VolumeList[i]==VolumeList[i-1]:
        OBV.append(OBV[i-1]+0)
        continue
    if VolumeList[i]<VolumeList[i-1]:
        OBV.append(OBV[i-1]-VolumeList[i])
        continue  
df['OBV'] = OBV

#Module 4 - Calculate Linear Regressions on Close and OBV plus errors

delay=5 # How many trading days to calculate moving slopes

OBV_regression = [0 for x in range(delay)]
OBV_regression_err = [0 for x in range(delay)]
Close_regression = [0 for x in range(delay)]
Close_regression_err = [0 for x in range(delay)]

Date = df['Date'].tolist()
x = np.array([x for x in range(delay)])
for i in range(delay,len(OBV)):
    y = df['OBV'].iloc[i-delay:i].values
    y2 = df['Close'].iloc[i-delay:i].values
    slope, intercept, r_value, p_value, err = stats.linregress(x,y)
    slope2, intercept2, r_value2, p_value2, err2 = stats.linregress(x,y2)
    #value = LinearRegression().fit(x,y).coef_[0]
    OBV_regression.append(slope)
    OBV_regression_err.append(err)
    Close_regression.append(slope2)
    Close_regression_err.append(err2)

df['OBV_Regression'] = OBV_regression
df['OBV_Regression_err'] = OBV_regression_err
df['Close_Regression'] = Close_regression
df['Close_Regression_err'] = Close_regression_err


In [4]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,Name,Change,ChangeAverage+,ChangeAverage-,RSI,OBV,OBV_Regression,OBV_Regression_err,Close_Regression,Close_Regression_err
0,2021-07-26,650.969971,668.200012,647.109985,657.619995,657.619995,25336600,TSLA,NaN,NaN,NaN,NaN,25336600,0.0,0.000000e+00,0.000000,0.000000
1,2021-07-27,663.400024,666.500000,627.239990,644.780029,644.780029,32813300,TSLA,-12.839966,NaN,NaN,NaN,58149900,0.0,0.000000e+00,0.000000,0.000000
2,2021-07-28,647.000000,654.969971,639.400024,646.979980,646.979980,16006600,TSLA,2.199951,NaN,NaN,NaN,42143300,0.0,0.000000e+00,0.000000,0.000000
3,2021-07-29,649.789978,683.690002,648.799988,677.349976,677.349976,30394600,TSLA,30.369995,NaN,NaN,NaN,72537900,0.0,0.000000e+00,0.000000,0.000000
4,2021-07-30,671.760010,697.530029,669.000000,687.200012,687.200012,29600500,TSLA,9.850037,NaN,NaN,NaN,42937400,0.0,0.000000e+00,0.000000,0.000000
5,2021-08-02,700.000000,726.940002,698.400024,709.669983,709.669983,33615800,TSLA,22.469971,NaN,NaN,NaN,76553200,4958960.0,5.866664e+06,9.172998,4.349127
6,2021-08-03,719.000000,722.650024,701.010010,709.739990,709.739990,21620300,TSLA,0.070007,NaN,NaN,NaN,54932900,3760070.0,5.451152e+06,16.999994,2.224623
7,2021-08-04,711.000000,724.900024,708.929993,710.919983,710.919983,17002600,TSLA,1.179993,NaN,NaN,NaN,37930300,2959450.0,5.643734e+06,15.784003,2.775824
8,2021-08-05,716.000000,720.950012,711.409973,714.630005,714.630005,12919600,TSLA,3.710022,NaN,NaN,NaN,25010700,-5721970.0,5.360427e+06,8.967999,2.415176
9,2021-08-06,711.900024,716.330017,697.630005,699.099976,699.099976,15576200,TSLA,-15.530029,NaN,NaN,NaN,40586900,-7447630.0,5.665174e+06,5.610999,2.336756
